# AutoRAG - DataCreation Pipeline

# Module

In [1]:
import warnings
warnings.filterwarnings(action='ignore')
import os
import sys
import dotenv
import nest_asyncio
dotenv.load_dotenv()
# nest_asyncio 적용
nest_asyncio.apply()

import pandas as pd
from glob import glob

# AutoRAG
from autorag.parser import Parser
from autorag.chunker import Chunker
from autorag.data.qa.schema import Raw, Corpus
from autorag.data.qa.filter.dontknow import dontknow_filter_rule_based
from autorag.data.qa.generation_gt.llama_index_gen_gt import make_basic_gen_gt,make_concise_gen_gt
from autorag.data.qa.query.llama_gen_query import factoid_query_gen
from autorag.data.qa.sample import random_single_hop

# llamaindex
from llama_index.llms.openai import OpenAI

# RawData Parse

In [55]:
file_paths = glob("../data/nursing/*")

In [48]:
idx = 16
print(file_paths[idx])

/home/livin/rag_pipeline/AutoRAG/data/nursing/14장 호흡기장애 대상자 간호.pdf


In [112]:
# parser 정의
parser = Parser(data_path_glob="/home/livin/rag_pipeline/AutoRAG/data/test/*", project_dir="../results/parse/test")
parser.start_parsing("../configs/Parse.yaml")

[10/11/24 16:50:50] INFO     [parser.py:33] >> Parsing Start...                                        ]8;id=240537;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py\parser.py]8;;\:]8;id=599190;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/parser.py#33\33]8;;\

                    INFO     [base.py:19] >> Running parser - langchain_parse module...                  ]8;id=656307;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/parse/base.py\base.py]8;;\:]8;id=987080;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/parse/base.py#19\19]8;;\

KeyError: 'upstagelayoutanalysis'

In [109]:
a = pd.read_parquet(f"../results/parse/test/4/0.parquet")

In [111]:
a["texts"][0]

"<header id='0' style='font-size:22px'>3</header><p id='1' data-category='paragraph' style='font-size:14px'>• 오디오 등을 장착하거나 차량 전면부에 판금<br>이나 도장을 하는 경우에 안전벨트 프리텐셔<br>너 작동에 영향을 미칠 수 있습니다.</p><br><p id='2' data-category='paragraph' style='font-size:14px'>이럴 경우 당사 직영 하이테크센터나 블루핸<br>즈에서 상담을 받으십시오. 안전벨트 프리텐<br>셔너가 제대로 작동하지 않으면, 에어백 경<br>고등(=)이 켜집니다.</p><br><h1 id='3' style='font-size:14px'>• 다음의 상황이 발생하면 당사 직영 하이테크<br>센터나 블루핸즈에서 신속하게 점검을 받으<br>십시오.</h1><br><p id='4' data-category='list' style='font-size:14px'>- 시동 'ON' 했을 때 에어백 경고등이 켜지<br>지 않는 경우<br>- 에어백 경고등이 시동 'ON' 했을 때 3-6초<br>이후에도 계속 켜져 있는 경우<br>- 시동 'ON' 했을 때 에어백 경고등이 깜빡<br>이는 경우<br>- 주행 중에 에어백 경고등이 켜질 경우</p><h1 id='5' style='font-size:18px'>(cid:1155)(cid:1162)</h1><br><p id='6' data-category='list' style='font-size:14px'>• 안전벨트 프리텐셔너가 작동하면, 큰 소음이<br>날 뿐만 아니라 연기 같은 미세한 가루가 보<br>일 수 있습니다. 이 가루는 독성이 없는 가스<br>입자이지만 피부에 자극을 줄 수도 있습니<br>다. 가루를 들이마시지 말고 손과 얼굴을 깨<br>끗이 닦으십시오.<br>• 안전벨트 프리텐셔너는 작동할 때 뜨거운 열<br>을 내므로 작동한 후 몇 분 동안은 안전벨트<br>프리텐셔너를 

In [81]:
# Integration
file_paths = glob("../data/nursing/*")
file_paths = [[i, n] for n,i in enumerate(file_paths)]
file_paths = sorted(file_paths, key=lambda x: int(x[0].split("/")[-1][:2]))

# nursing 폴더에 있는 parquet을 하나씩 읽어서 통합 parquet만들기

df = pd.DataFrame()
for file_path in file_paths:
    df = pd.concat([df, pd.read_parquet(f"../results/parse/nursing/{file_path[1]}/0.parquet")])
    # print(file_path[0])

df.index = range(len(df))
# df.to_parquet("../results/parse/nursing/nursing.parquet")


In [89]:
parse_df = pd.read_parquet("../results/parse/nursing/nursing.parquet")
parse_df

,texts,path,page,last_modified_datetime
0,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
1,<h1 id='0' style='font-size:18px'>강의용 자료파일 사용 ...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
2,<h1 id='0' style='font-size:18px'>강의용 자료파일 사용 ...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
3,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
4,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
5,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
6,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
7,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
8,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11
9,<p id='0' data-category='list'></p><figure><im...,/home/livin/rag_pipeline/AutoRAG/data/nursing/...,-1,2024-10-11


In [121]:
parse_df.iloc[3]["texts"]

'<p id=\'0\' data-category=\'list\'></p><figure><img id=\'1\' alt="" data-coord="top-left:(57,617); bottom-right:(1308,1044)" /></figure><figure><img id=\'2\' alt="" data-coord="top-left:(57,59); bottom-right:(672,485)" /></figure><p id=\'3\' data-category=\'list\'></p><figure><img id=\'4\' alt="" data-coord="top-left:(118,214); bottom-right:(1293,548)" /></figure><figure><img id=\'5\' alt="" data-coord="top-left:(187,1155); bottom-right:(646,1598)" /></figure><br><figure><img id=\'6\' alt="" data-coord="top-left:(674,1205); bottom-right:(1231,1583)" /></figure><figure><img id=\'7\' alt="" data-coord="top-left:(1144,57); bottom-right:(1314,157)" /></figure><figure><img id=\'8\' alt="" data-coord="top-left:(151,206); bottom-right:(1199,1152)" /></figure><figure><img id=\'9\' alt="" data-coord="top-left:(731,1216); bottom-right:(1159,1463)" /></figure><figure><img id=\'10\' alt="" data-coord="top-left:(162,221); bottom-right:(1210,1171)" /></figure><figure><img id=\'11\' alt="" data-coor

# Chunking

In [2]:
chunker = Chunker.from_parquet(parsed_data_path="../results/parse/nursing/nursing_corpus.parquet", project_dir="../results/chunk/nursing")
chunker.start_chunking("../configs/Chunk.yaml")

[10/16/24 11:41:51] INFO     [chunker.py:48] >> Chunking Start...                                     ]8;id=438389;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=689977;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py#48\48]8;;\

                    INFO     [base.py:22] >> Running chunker - llama_index_chunk module...               ]8;id=445841;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/chunk/base.py\base.py]8;;\:]8;id=943714;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/data/chunk/base.py#22\22]8;;\

[10/16/24 11:41:58] INFO     [chunker.py:55] >> Chunking Done!                                        ]8;id=35710;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py\chunker.py]8;;\:]8;id=363659;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/autorag/chunker.py#55\55]8;;\

In [5]:
# Chunk 결과 확인
corpus_df = pd.read_parquet("../results/chunk/nursing/2_nursing_chunk/0.parquet")
corpus_df

,doc_id,contents,path,start_end_idx,metadata
0,1f2998c5-c672-4fc3-b83f-0ff60fdd13ac,파일 제목: 00 도입부(하권)\n 내용: <h1 id='0' styl...,00 도입부(하권),"[0, 1611]",{'last_modified_datetime': 2024-10-16 11:38:25...
1,c4e804e5-2416-483c-9709-b2e00adfdb56,파일 제목: 00 도입부(하권)\n 내용: 가능함)</p><figure...,00 도입부(하권),"[1273, 2526]",{'last_modified_datetime': 2024-10-16 11:38:25...
2,07282686-a235-4362-ae78-8d045208f638,파일 제목: 00 도입부(하권)\n 내용: 온라<br>인과 오프라인 교...,00 도입부(하권),"[2278, 3740]",{'last_modified_datetime': 2024-10-16 11:38:25...
3,48d2edeb-19fb-49f4-90cb-d86f73788222,파일 제목: 00 도입부(하권)\n 내용: style='font-siz...,00 도입부(하권),"[3320, 4666]",{'last_modified_datetime': 2024-10-16 11:38:25...
4,b26dbd0e-4110-4443-9a3b-5fcdd86235e0,파일 제목: 00 도입부(하권)\n 내용: 결정에 관여하고 자기간호에 ...,00 도입부(하권),"[4413, 5908]",{'last_modified_datetime': 2024-10-16 11:38:25...
...,...,...,...,...,...
3044,92ff5a3f-b728-4739-a53b-77742fd50bb7,"파일 제목: 27장 화상간호\n 내용: 받으므로, 외래 방<br>문 ...",27장 화상간호,"[69394, 70717]",{'last_modified_datetime': 2024-10-16 11:38:25...
3045,2585466f-9274-457e-8a95-7aa157f99ad7,파일 제목: 27장 화상간호\n 내용: 요양시설)의 요구를 사정한다....,27장 화상간호,"[70318, 71558]",{'last_modified_datetime': 2024-10-16 11:38:25...
3046,8192d4aa-e4ea-4967-886c-4d6117f5825a,파일 제목: 27장 화상간호\n 내용: 잠재적 성문제에 대한 질문에 ...,27장 화상간호,"[71351, 72534]",{'last_modified_datetime': 2024-10-16 11:38:25...
3047,032efa24-b7a1-44bd-858f-ea5021ecd34c,"파일 제목: 27장 화상간호\n 내용: 의뢰하고, 오랜기간 직업훈련을...",27장 화상간호,"[72214, 73176]",{'last_modified_datetime': 2024-10-16 11:38:25...


In [6]:
corpus_df.iloc[2]

doc_id                        07282686-a235-4362-ae78-8d045208f638
contents         파일 제목: 00 도입부(하권)\n 내용: 온라<br>인과 오프라인 교...
path                                             00 도입부(하권)
start_end_idx                                         [2278, 3740]
metadata         {'last_modified_datetime': 2024-10-16 11:38:25...
Name: 2, dtype: object

# QA Creation

In [24]:
# load RAW data
raw_df = pd.read_parquet("../results/chunk/nursing/2_nursing_chunk/0.parquet")
raw_instance = Raw(raw_df)

# load Corpus data
corpus_df = pd.read_parquet("../results/chunk/nursing/2_nursing_chunk/0.parquet")
corpus_instance = Corpus(corpus_df, raw_instance)

In [25]:
llm = OpenAI(model="gpt-4o-mini")
for i in range(10,12):
    random_state = 7*i
    print(random_state)
    initial_qa = (
        corpus_instance.sample(random_single_hop, random_state=random_state, n=30) # 몇개의 corpus를 뽑을지 결정
        .map(
            lambda df: df.reset_index(drop=True), # Drop index
        )
        .make_retrieval_gt_contents() # retrieval_gt_contents 생성
        .batch_apply(
            factoid_query_gen,  # query generation
            llm=llm,
            lang="ko",
        )
        .batch_apply(
            make_basic_gen_gt,  # answer generation (basic)
            llm=llm,
            lang="ko",
        )
        .filter(
            dontknow_filter_rule_based,  # unanswerable filter
            lang="ko",
        )
    )

    # 생성된 qa 저장
    initial_qa.to_parquet(f'../results/qa/nursing/qa_{i}.parquet', '../results/qa/nursing/corpus.parquet')

70


[10/16/24 12:01:58] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=709129;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=432813;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"                 

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 128000 tokens. However, your messages resulted in 152421 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=916323;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=947258;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"                 

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=451295;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=599127;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"                 

...

[10/16/24 12:01:59] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=343898;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=58686;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"                 

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=701736;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=750468;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"                 

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=764950;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=672350;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"                 

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=543509;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=301143;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"                 

[10/16/24 12:02:01] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=99581;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=756577;file:///home/livin/anaconda3/envs/rag_pipeline/lib/python3.11/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"                 

In [14]:
# 생성된 qa 저장
initial_qa.to_parquet('../results/qa/nursing/qa_1.parquet', '../results/qa/nursing/corpus.parquet')

In [9]:
# 생성된 qa 확인
initial_qa.data.head(2)

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,48254d55-e768-4e8a-ad38-35afffa6322e,[[b0aaa5b5-e3dd-4f5b-a50f-3d4ae3724109]],[[파일 제목: 19장 남성 생식기계 대상자 간호...,고환암은 주로 몇 세의 남성에게 많이 발생합니까?,[고환암은 주로 15~40세의 남성에게 많이 발생합니다.]
1,32f6f1ee-7b61-4a63-b581-c7dd97f0ca7d,[[48380b63-cd3a-4873-91ca-d14fd6230e4f]],[[파일 제목: 22장 신경계장애 대상자간호-(2)\...,삼차신경통의 통증은 일반적으로 어느 쪽 얼굴에만 발생합니까?,[삼차신경통의 통증은 일반적으로 한쪽 얼굴에만 발생합니다.]


In [16]:
# Chunk 결과 확인
corpus_df = pd.read_parquet("/home/livin/rag_pipeline/AutoRAG/results/qa/hyundai_upstage_qa_3.parquet")
corpus_df.head(2)

,qid,query,retrieval_gt,generation_gt
0,052b323b-fa7f-41f0-9dd6-41791d2a5d6f,에어컨을 사용한 후 차량 밑바닥에서 물이 떨어지는 이유는 무엇인가요?,[[5d54c7a1-134d-4c29-a07f-68f68d7979a8]],[에어컨을 사용한 후 차량 밑바닥에서 물이 떨어지는 것은 정상적인 현상입니다.]
1,3e372364-e950-41bb-beff-dad5cecfdedd,"USB 충전 단자는 어떤 용도로 사용되며, 이 단자를 통해 미디어를 재생할 수 있습니까?",[[596ccf0f-1e57-41dc-908a-a3f99803ba80]],"[USB 충전 단자는 충전 전용으로 사용되며, 이 단자를 통해서는 인포테인먼트 시스..."


# QA Integration
* 다양한 QA 셋을 통합하여 하나의 QA 셋으로 만듦

In [17]:
qa_list = ["../results/qa/hyundai_upstage_qa_1.parquet", "../results/qa/hyundai_upstage_qa_2.parquet", "../results/qa/hyundai_upstage_qa_3.parquet", "../results/qa/hyundai_upstage_qa_4.parquet"]

In [19]:
# 여러 qa 파일을 하나의 qa 파일로 합치기
qa_df = pd.concat([pd.read_parquet(qa) for qa in qa_list], ignore_index=True)

# qa 저장
qa_df.to_parquet("../results/qa/hyundai_upstage_qa_1234.parquet")

# Test

In [15]:
corpus_df = pd.read_parquet('../results/qa/nursing/corpus.parquet')

In [19]:
re = []
for i in range(len(corpus_df)):
    re.append(len(corpus_df.iloc[i]["contents"]))

In [23]:
min(re), max(re)

(343, 2104)